# Data Analytics - ML25M 

### Imports

In [ ]:
import torch
import time
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from torch.utils.tensorboard import SummaryWriter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import itertools
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use. 
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower

fix_random(42)


## Data Acquisition

movies.csv contains the following fields:

* movieId - a unique identifier for each movie.
* title - the title of the movie.
* genres - a pipe-separated list of genres for the movie.

It will be used to get the movie title and the genres.

In [ ]:
movies_df = pd.read_csv('./ml-25m/movies.csv')
genres = movies_df['genres'].str.get_dummies(sep='|')
movies_df = pd.concat([movies_df, genres], axis=1)
movies_df.drop('genres', axis=1, inplace=True)
movies_df.head(3)


genome-scores.csv contains the following fields:

* movieId - a unique identifier for each movie.
* tagId - a unique identifier for each tag.
* relevance - a score from 0.0 to 1.0 representing the relevance of the tag to the movie.

Combined with the tags.csv file, this will be used to assign tags and their relevance to each movie.

In [ ]:
scores_df = pd.read_csv('./ml-25m/genome-scores.csv')
tags_df = pd.read_csv('./ml-25m/genome-tags.csv')
ratings_df = pd.read_csv('./ml-25m/ratings.csv')

df = movies_df.merge(scores_df, on='movieId')
df = df.merge(tags_df, on='tagId')
df = df.pivot_table(index=['movieId', 'title'], columns='tag', values='relevance', fill_value=0).reset_index().rename_axis(None, axis=1)

# average rating for each movie
ratings_df = ratings_df.groupby(['movieId'])['rating'].mean().reset_index()
# round ratings to the nearest 0.5
ratings_df['rating'] = ratings_df['rating'].apply(lambda x: round(x * 2) / 2)

# # mode rating for each movie
# ratings_df = ratings_df.groupby(['movieId'])['rating'].agg(lambda x: x.value_counts().index[0]).reset_index()

df = ratings_df.merge(df, on='movieId')

# movieId and title are not needed for the model
df.drop(['movieId', 'title'], axis=1, inplace=True)

## Data visualization

In [ ]:
print(f'Number of samples: {df.shape[0]}')
df.head(3)

In [ ]:
# Ratings distribution
sns.countplot(x='rating', data=df, palette='flare')
plt.title('Ratings distribution')
for p in plt.gca().patches:
    plt.gca().annotate('{:.0f}'.format(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.show()

# mean, median, and standard deviation of ratings
print("Mode: ", df['rating'].mode())
print('Median: {:.2f}'.format(df['rating'].median()))
print('Std: {:.2f}'.format(df['rating'].std()))

## Data Preprocessing

In [ ]:
N_LABELS = df.rating.nunique()
print(f'Number of labels: {N_LABELS}')

X = df.drop('rating', axis=1)
Y = df['rating']

# encode Y
Y = LabelEncoder().fit_transform(Y)

lda = LinearDiscriminantAnalysis()
lda.fit(X, Y)
X = lda.transform(X)


# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f'Number of training samples: {X_train.shape}')
print(f'Number of testing samples: {X_test.shape}')

results = pd.DataFrame(columns=['Model', 'Accuracy'])

## Modeling

### Random Forest Classifier

In [ ]:
# Create the model with 100 trees
random_forest = RandomForestClassifier(n_estimators=50,
                                bootstrap = True,
                                max_features = 'sqrt')

# Fit on training data
random_forest.fit(X_train, Y_train)

# accuracy
accuracy = random_forest.score(X_test, Y_test)
print('Accuracy:', accuracy)

results = pd.concat([results, pd.DataFrame({'Model': 'Random Forest', 'Accuracy': accuracy}, index=[0])], ignore_index=True)

### Logistic Classifier

In [ ]:
logistic_classifier = LogisticRegression()
logistic_classifier.fit(X_train, Y_train)

# accuracy
accuracy = logistic_classifier.score(X_test, Y_test)
print('Accuracy:', accuracy)

results = pd.concat([results, pd.DataFrame({'Model': 'Logistic Regression', 'Accuracy': accuracy}, index=[0])], ignore_index=True)

### Naive Bayes Classifier

In [ ]:
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, Y_train)

# accuracy
accuracy = naive_bayes.score(X_test, Y_test)
print('Accuracy:', accuracy)

results = pd.concat([results, pd.DataFrame({'Model': 'Naive Bayes', 'Accuracy': accuracy}, index=[0])], ignore_index=True)

### KNN Classifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, Y_train)

# accuracy
accuracy = knn.score(X_test, Y_test)
print('Accuracy:', accuracy)

results = pd.concat([results, pd.DataFrame({'Model': 'KNN', 'Accuracy': accuracy}, index=[0])], ignore_index=True)

### SVM Classifier

In [ ]:
svm = SVC()
svm.fit(X_train, Y_train)

# accuracy
accuracy = svm.score(X_test, Y_test)
print('Accuracy:', accuracy)

results = pd.concat([results, pd.DataFrame({'Model': 'SVM', 'Accuracy': accuracy}, index=[0])], ignore_index=True)

### Neural Network

In [ ]:
# PyTorch Device
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print('Using device:', device)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

print(f'Number of training samples: {X_train.shape[0]}')
print(f'Number of validation samples: {X_val.shape[0]}')

# train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(Y_train, dtype=torch.long)), batch_size=batch, shuffle=True)
val_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(Y_val, dtype=torch.long)), batch_size= 64, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(Y_test, dtype=torch.long)), batch_size= 64, shuffle=True)

In [ ]:
# Effective Number of Samples
def get_weights_effective_num_of_samples(no_of_classes, beta, samples_per_cls):
    effective_num = 1.0 - np.power(beta, samples_per_cls)
    class_weights = (1.0 - beta) / np.array(effective_num)
    class_weights = class_weights / np.sum(class_weights)*no_of_classes
    return {i: w for i, w in enumerate(class_weights)}

weights = get_weights_effective_num_of_samples(N_LABELS, 0.999, np.bincount(Y_train))

In [ ]:
def get_nn_model(input_size, hidden_size, num_classes, dropout_prob=0, depth=1):
    model = [
        torch.nn.Linear(input_size, hidden_size),
        torch.nn.ReLU(),
        torch.nn.Dropout(dropout_prob)
    ]

    for i in range(depth):
        model.append(torch.nn.Linear(hidden_size, hidden_size))
        model.append(torch.nn.ReLU())
        model.append(torch.nn.Dropout(dropout_prob))

    model.append(torch.nn.Linear(hidden_size, num_classes))
    model.append(torch.nn.Softmax(dim=1))

    return torch.nn.Sequential(*model)

In [ ]:
# hyperparameters
hidden_sizes = [32, 64, 128, 256]
nums_epochs = [50, 100]
depth = [2, 3, 4]
batch_sizes = [8, 16, 32, 64, 128]
learning_rate = [0.01]
step_size_lr_decay = [10, 25, 50]
momentum = [0, 0.99]
dropout_prob = 0.2
patience = 10

hyperparameters = itertools.product(hidden_sizes, depth, nums_epochs, batch_sizes, learning_rate, step_size_lr_decay, momentum)

In [ ]:
# training process
def train_model(model, criterion, optimizer, scheduler, epochs, data_loader, val_loader, device, writer):
    n_iter = 0

    best_val_loss = float('inf')
    epochs_since_last_improvement = 0
 
    start = time.time()

    for epoch in range(epochs):
        model.train()

        start_epoch = time.time()

        for data, targets in data_loader:
            data, targets = data.to(device), targets.to(device) 

            optimizer.zero_grad()

            # Forward pass
            y_pred = model(data)

            # Compute Loss
            loss = criterion(y_pred, targets)
            writer.add_scalar("Loss/train", loss, n_iter)

            # Backward pass
            loss.backward()
            optimizer.step()

            n_iter += 1

        scheduler.step()

        # Compute Val Loss
        labels, _, y_pred = test_model(model, val_loader, device)
        val_loss = criterion(y_pred, labels)
        writer.add_scalar("Loss/val", val_loss, epoch)

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'models/best_model.pth')
            epochs_since_last_improvement = 0
        else:
            epochs_since_last_improvement += 1

        if epochs_since_last_improvement >= patience:
            break

        print('Epoch [{}/{}] - {:.2f} seconds - loss: {:.4f} - val_loss: {:.4f} - patience: {}'.format(epoch+1, epochs, time.time() - start_epoch, loss.item(), val_loss.item(), epochs_since_last_improvement), end='\r')
        
    print('\nTraining ended after {:.2f} seconds'.format(time.time() - start))

    # Restore best model
    model.load_state_dict(torch.load('models/best_model.pth'))
    return model

# evaluation process
def test_model(model, data_loader, device):
    model.eval()
    y_pred = []
    y_test = []

    with torch.no_grad():
        for data, targets in data_loader:
            data, targets = data.to(device), targets.to(device)

            y_pred.append(model(data))
            y_test.append(targets)

    y_pred = torch.cat(y_pred, dim=0)
    y_test = torch.cat(y_test, dim=0)

    y_pred_c = torch.argmax(y_pred, dim=1)

    return y_test, y_pred_c, y_pred


In [ ]:
best_acc = 0
best_model = None

for hidden_size, depth, num_epochs, batch, lr, step_size, momentum in hyperparameters:
    
    if not os.path.exists('models'):
        os.makedirs('models')

    print("\nTraining with hidden_size={}, depth={}, num_epochs={}, batch={}, lr={}, step_size={}, momentum={}".format(hidden_size, depth, num_epochs, batch, lr, step_size, momentum))
    log_name = "dim:"+str(hidden_size)+"_depth:"+str(depth)+"_epochs:"+str(num_epochs)+"_batch:"+str(batch)+"_lr:"+str(lr)+"_step_size:"+str(step_size)+"_momentum:"+str(momentum)
    
    if os.path.exists('runs/'+log_name):
        print("Model already trained, skipping...")
        continue

    # start tensorboard
    writer = SummaryWriter('runs/'+log_name)

    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(Y_train, dtype=torch.long)), batch_size=batch, shuffle=True)

    model = get_nn_model(X_train.shape[1], hidden_size, N_LABELS, dropout_prob, depth=depth)
    model.to(device)

    criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor(list(weights.values()), dtype=torch.float32).to(device))
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

    # train
    model = train_model(model, criterion, optimizer, scheduler, num_epochs, train_loader, train_loader, device, writer)

    # validate
    y_test, y_pred_c, y_pred = test_model(model, test_loader, device)

    metrics = classification_report(y_test.cpu(), y_pred_c.cpu(), output_dict=True, zero_division=0)
    writer.add_scalar('metrics/test accuracy', metrics['accuracy'])

    writer.add_hparams({'hidden_size': hidden_size, 'depth': depth, 'num_epochs': num_epochs, 'batch': batch, 'lr': lr, 'step_size': step_size, 'momentum': momentum}, {'hparam/accuracy': metrics['accuracy']})

    if metrics['accuracy'] > best_acc:
        best_acc = metrics['accuracy']
        best_model = model
        torch.save(model.state_dict(), 'best_model.pth')

    
    torch.save(model.state_dict(), 'models/'+log_name+'.pth')

    writer.flush()

writer.close()

In [ ]:
# test
y_test, y_pred_c, y_pred = test_model(best_model, test_loader, device)
print(classification_report(y_test.cpu(), y_pred_c.cpu()))
accuracy = accuracy_score(y_test.cpu(), y_pred_c.cpu())

results = pd.concat([results, pd.DataFrame({'Model': 'NeuralNetwork', 'Accuracy': accuracy}, index=[0])], ignore_index=True)

### Deep Learning for Tabular Data

[PyTorch Tabular](https://github.com/manujosephv/pytorch_tabular#installation)

## Evaluation